# Thermal Dynamics Analysis

## Objectives:
1. Analyze thermal dynamics for each room (heat-up, cool-down rates)
2. Estimate thermal parameters (resistance, capacitance, time constants)
3. Model room-to-room heat transfer and coupling
4. Create thermal models for predictive control
5. Identify thermal efficiency improvements

## Key Analyses:
- Heat-up and cool-down curve fitting
- Thermal resistance and capacitance estimation (RC model)
- Solar gain and internal heat gain analysis
- Room coupling and heat transfer analysis
- Thermal comfort analysis (temperature stability)
- Energy efficiency metrics per room
- Predictive thermal model validation

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import asyncio
import warnings
warnings.filterwarnings('ignore')

# Add pems_v2 directory to path for imports
sys.path.append(str(Path('../pems_v2').resolve()))

# Import project modules
from analysis.core.data_extraction import DataExtractor
from analysis.analyzers.thermal_analysis import ThermalAnalyzer
from config.settings import PEMSSettings
from config.energy_settings import ROOM_CONFIG

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load Thermal Data

Load room temperature, relay states, and weather data for thermal analysis

In [ ]:
# Initialize settings and extractors
settings = PEMSSettings()
extractor = DataExtractor(settings)
thermal_analyzer = ThermalAnalyzer()

# Define analysis period (last 90 days for comprehensive thermal analysis)
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

print(f"Analysis period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

In [ ]:
# Extract thermal data
async def load_thermal_data():
    """Load thermal analysis data."""
    print("Loading room temperature data...")
    room_data = await extractor.extract_room_temperatures(start_date, end_date)
    
    print("Loading heating relay data...")
    relay_data = await extractor.extract_relay_states(start_date, end_date)
    
    print("Loading weather data...")
    weather_data = await extractor.extract_weather_data(start_date, end_date)
    current_weather = await extractor.extract_current_weather(start_date, end_date)
    
    print("Loading PV data for solar gains...")
    pv_data = await extractor.extract_pv_data(start_date, end_date)
    
    return room_data, relay_data, weather_data, current_weather, pv_data

# Load data
room_data, relay_data, weather_data, current_weather, pv_data = await load_thermal_data()

print(f"\nData loaded:")
print(f"  Room temperature data: {len(room_data)} rooms")
print(f"  Relay data: {len(relay_data)} rooms")
print(f"  Weather records: {len(weather_data)}")
print(f"  Current weather records: {len(current_weather)}")
print(f"  PV records: {len(pv_data)}")

# Display available rooms
common_rooms = set(room_data.keys()) & set(relay_data.keys())
print(f"\nRooms with both temperature and relay data: {list(common_rooms)}")

## 2. Merge and Prepare Thermal Data

Combine temperature, heating, and weather data for thermal analysis

In [ ]:
# Merge weather data sources
merged_weather = pd.DataFrame()
if not weather_data.empty:
    merged_weather = weather_data.copy()

if not current_weather.empty:
    # Merge current weather data
    if merged_weather.empty:
        merged_weather = current_weather.copy()
    else:
        merged_weather = pd.merge(merged_weather, current_weather, 
                                 left_index=True, right_index=True, 
                                 how='outer', suffixes=('', '_current'))

print(f"Merged weather data shape: {merged_weather.shape}")
if not merged_weather.empty:
    print(f"Weather columns: {list(merged_weather.columns)}")

# Prepare thermal datasets for each room
thermal_datasets = {}

for room in common_rooms:
    room_temp = room_data[room]
    room_relay = relay_data[room]
    
    if not room_temp.empty and not room_relay.empty and 'temperature' in room_temp.columns:
        # Resample to consistent 15-minute intervals
        temp_15min = room_temp['temperature'].resample('15T').mean()
        relay_15min = room_relay['value'].resample('15T').mean()
        
        # Combine room data
        room_thermal = pd.DataFrame({
            'temperature': temp_15min,
            'heating_state': relay_15min
        })
        
        # Add weather data
        if not merged_weather.empty:
            weather_15min = merged_weather.resample('15T').mean()
            room_thermal = pd.merge(room_thermal, weather_15min, 
                                   left_index=True, right_index=True, how='left')
        
        # Add solar radiation from PV data if available
        if not pv_data.empty and 'InputPower' in pv_data.columns:
            solar_15min = pv_data['InputPower'].resample('15T').mean()
            room_thermal = pd.merge(room_thermal, solar_15min.to_frame('solar_power'), 
                                   left_index=True, right_index=True, how='left')
        
        # Remove rows with missing temperature data
        room_thermal = room_thermal.dropna(subset=['temperature'])
        
        # Add time features
        room_thermal['hour'] = room_thermal.index.hour
        room_thermal['day_of_year'] = room_thermal.index.dayofyear
        room_thermal['is_weekend'] = room_thermal.index.weekday.isin([5, 6])
        
        thermal_datasets[room] = room_thermal
        
        print(f"Room {room}: {len(room_thermal)} records prepared")

print(f"\nThermal datasets prepared for {len(thermal_datasets)} rooms")

## 3. Heat-up and Cool-down Analysis

Analyze heating and cooling curves to estimate thermal parameters

In [ ]:
def detect_heating_cycles(room_data, min_duration_hours=1, temp_change_threshold=0.5):
    """Detect distinct heating and cooling cycles."""
    
    # Detect state changes
    heating_changes = room_data['heating_state'].diff().fillna(0)
    
    # Find heating start/stop events
    heating_starts = room_data.index[heating_changes > 0.5]
    heating_stops = room_data.index[heating_changes < -0.5]
    
    cycles = []
    
    # Analyze heating cycles
    for start_time in heating_starts:
        # Find corresponding stop time
        stop_times = heating_stops[heating_stops > start_time]
        if len(stop_times) > 0:
            stop_time = stop_times[0]
            duration = (stop_time - start_time).total_seconds() / 3600  # hours
            
            if duration >= min_duration_hours:
                # Extract cycle data
                cycle_data = room_data.loc[start_time:stop_time].copy()
                
                if len(cycle_data) > 4:  # Need sufficient data points
                    temp_start = cycle_data['temperature'].iloc[0]
                    temp_end = cycle_data['temperature'].iloc[-1]
                    temp_change = temp_end - temp_start
                    
                    if temp_change >= temp_change_threshold:
                        cycles.append({
                            'type': 'heating',
                            'start_time': start_time,
                            'end_time': stop_time,
                            'duration_hours': duration,
                            'temp_start': temp_start,
                            'temp_end': temp_end,
                            'temp_change': temp_change,
                            'data': cycle_data
                        })
    
    # Analyze cooling cycles (heating off periods)
    for stop_time in heating_stops:
        # Find next heating start
        start_times = heating_starts[heating_starts > stop_time]
        if len(start_times) > 0:
            next_start = start_times[0]
            duration = (next_start - stop_time).total_seconds() / 3600  # hours
            
            if duration >= min_duration_hours:
                # Extract cycle data
                cycle_data = room_data.loc[stop_time:next_start].copy()
                
                if len(cycle_data) > 4:  # Need sufficient data points
                    temp_start = cycle_data['temperature'].iloc[0]
                    temp_end = cycle_data['temperature'].iloc[-1]
                    temp_change = temp_end - temp_start
                    
                    if abs(temp_change) >= temp_change_threshold:
                        cycles.append({
                            'type': 'cooling',
                            'start_time': stop_time,
                            'end_time': next_start,
                            'duration_hours': duration,
                            'temp_start': temp_start,
                            'temp_end': temp_end,
                            'temp_change': temp_change,
                            'data': cycle_data
                        })
    
    return cycles

# Detect heating/cooling cycles for each room
room_cycles = {}

for room, room_data in thermal_datasets.items():
    cycles = detect_heating_cycles(room_data)
    room_cycles[room] = cycles
    
    heating_cycles = [c for c in cycles if c['type'] == 'heating']
    cooling_cycles = [c for c in cycles if c['type'] == 'cooling']
    
    print(f"Room {room}: {len(heating_cycles)} heating cycles, {len(cooling_cycles)} cooling cycles")

print(f"\nCycle detection completed for {len(room_cycles)} rooms")

In [ ]:
def fit_exponential_curve(time_hours, temperature, curve_type='heating'):
    """Fit exponential curve to heating/cooling data."""
    
    if len(time_hours) < 4 or len(temperature) < 4:
        return None
        
    try:
        # Normalize time to start from 0
        t = time_hours - time_hours[0]
        T = temperature.values if hasattr(temperature, 'values') else temperature
        
        T_start = T[0]
        
        if curve_type == 'heating':
            # Heating curve: T(t) = T_final - (T_final - T_start) * exp(-t/tau)
            # Rearrange to: T(t) = T_start + (T_final - T_start) * (1 - exp(-t/tau))
            
            def heating_curve(t, T_final, tau):
                return T_start + (T_final - T_start) * (1 - np.exp(-t / tau))
            
            # Initial guess
            T_final_guess = max(T) + 2  # Assume target slightly above max observed
            tau_guess = len(t) * 0.25  # Initial time constant guess
            
            popt, pcov = optimize.curve_fit(heating_curve, t, T, 
                                          p0=[T_final_guess, tau_guess],
                                          bounds=([T_start, 0.1], [50, 24]),
                                          maxfev=1000)
            
            T_final_fit, tau_fit = popt
            T_pred = heating_curve(t, T_final_fit, tau_fit)
            
        else:  # cooling
            # Cooling curve: T(t) = T_ambient + (T_start - T_ambient) * exp(-t/tau)
            
            def cooling_curve(t, T_ambient, tau):
                return T_ambient + (T_start - T_ambient) * np.exp(-t / tau)
            
            # Initial guess
            T_ambient_guess = min(T) - 1  # Assume ambient slightly below min observed
            tau_guess = len(t) * 0.25
            
            popt, pcov = optimize.curve_fit(cooling_curve, t, T,
                                          p0=[T_ambient_guess, tau_guess],
                                          bounds=([-10, 0.1], [T_start, 48]),
                                          maxfev=1000)
            
            T_ambient_fit, tau_fit = popt
            T_pred = cooling_curve(t, T_ambient_fit, tau_fit)
            
        # Calculate fit quality
        r2 = r2_score(T, T_pred)
        rmse = np.sqrt(mean_squared_error(T, T_pred))
        
        result = {
            'tau_hours': tau_fit,
            'r2': r2,
            'rmse': rmse,
            'params': popt,
            'time': t,
            'temp_actual': T,
            'temp_predicted': T_pred
        }
        
        if curve_type == 'heating':
            result['T_final'] = T_final_fit
        else:
            result['T_ambient'] = T_ambient_fit
            
        return result
        
    except Exception as e:
        print(f"Curve fitting failed: {e}")
        return None

# Fit curves for each room's cycles
room_thermal_params = {}

for room, cycles in room_cycles.items():
    heating_cycles = [c for c in cycles if c['type'] == 'heating']
    cooling_cycles = [c for c in cycles if c['type'] == 'cooling']
    
    heating_fits = []
    cooling_fits = []
    
    # Fit heating curves
    for cycle in heating_cycles[:5]:  # Analyze first 5 cycles
        data = cycle['data']
        time_hours = (data.index - data.index[0]).total_seconds() / 3600
        
        fit_result = fit_exponential_curve(time_hours, data['temperature'], 'heating')
        if fit_result and fit_result['r2'] > 0.7:  # Only keep good fits
            heating_fits.append(fit_result)
    
    # Fit cooling curves
    for cycle in cooling_cycles[:5]:  # Analyze first 5 cycles
        data = cycle['data']
        time_hours = (data.index - data.index[0]).total_seconds() / 3600
        
        fit_result = fit_exponential_curve(time_hours, data['temperature'], 'cooling')
        if fit_result and fit_result['r2'] > 0.7:  # Only keep good fits
            cooling_fits.append(fit_result)
    
    # Calculate average parameters
    room_params = {
        'heating_cycles_analyzed': len(heating_fits),
        'cooling_cycles_analyzed': len(cooling_fits)
    }
    
    if heating_fits:
        room_params['heating_tau_avg'] = np.mean([f['tau_hours'] for f in heating_fits])
        room_params['heating_tau_std'] = np.std([f['tau_hours'] for f in heating_fits])
        room_params['heating_r2_avg'] = np.mean([f['r2'] for f in heating_fits])
        room_params['heating_fits'] = heating_fits
    
    if cooling_fits:
        room_params['cooling_tau_avg'] = np.mean([f['tau_hours'] for f in cooling_fits])
        room_params['cooling_tau_std'] = np.std([f['tau_hours'] for f in cooling_fits])
        room_params['cooling_r2_avg'] = np.mean([f['r2'] for f in cooling_fits])
        room_params['cooling_fits'] = cooling_fits
    
    room_thermal_params[room] = room_params

# Display thermal parameters
print("\nThermal Time Constants Analysis:")
print("=" * 80)
print(f"{'Room':15s} {'Heat τ (h)':10s} {'Cool τ (h)':10s} {'Heat R²':8s} {'Cool R²':8s} {'Cycles':8s}")
print("-" * 80)

for room, params in room_thermal_params.items():
    heat_tau = f"{params.get('heating_tau_avg', 0):.2f}" if 'heating_tau_avg' in params else "N/A"
    cool_tau = f"{params.get('cooling_tau_avg', 0):.2f}" if 'cooling_tau_avg' in params else "N/A"
    heat_r2 = f"{params.get('heating_r2_avg', 0):.3f}" if 'heating_r2_avg' in params else "N/A"
    cool_r2 = f"{params.get('cooling_r2_avg', 0):.3f}" if 'cooling_r2_avg' in params else "N/A"
    cycles = f"{params.get('heating_cycles_analyzed', 0)}+{params.get('cooling_cycles_analyzed', 0)}"
    
    print(f"{room:15s} {heat_tau:10s} {cool_tau:10s} {heat_r2:8s} {cool_r2:8s} {cycles:8s}")

## 4. Visualize Thermal Curves

Plot representative heating and cooling curves with fitted models

In [ ]:
# Visualize thermal curves for rooms with good fits
rooms_with_good_fits = [room for room, params in room_thermal_params.items() 
                       if params.get('heating_cycles_analyzed', 0) > 0 or params.get('cooling_cycles_analyzed', 0) > 0]

if rooms_with_good_fits:
    # Select up to 6 rooms for visualization
    rooms_to_plot = rooms_with_good_fits[:6]
    
    fig, axes = plt.subplots(len(rooms_to_plot), 2, figsize=(15, 4*len(rooms_to_plot)))
    if len(rooms_to_plot) == 1:
        axes = axes.reshape(1, -1)
    
    for idx, room in enumerate(rooms_to_plot):
        params = room_thermal_params[room]
        
        # Plot heating curve
        if 'heating_fits' in params and len(params['heating_fits']) > 0:
            # Plot best heating fit
            best_heating = max(params['heating_fits'], key=lambda x: x['r2'])
            
            axes[idx, 0].plot(best_heating['time'], best_heating['temp_actual'], 
                             'b-', linewidth=2, label='Actual')
            axes[idx, 0].plot(best_heating['time'], best_heating['temp_predicted'], 
                             'r--', linewidth=2, label='Model')
            
            axes[idx, 0].set_title(f'{room} - Heating Curve\nτ={best_heating["tau_hours"]:.2f}h, R²={best_heating["r2"]:.3f}')
            axes[idx, 0].set_xlabel('Time (hours)')
            axes[idx, 0].set_ylabel('Temperature (°C)')
            axes[idx, 0].legend()
            axes[idx, 0].grid(True, alpha=0.3)
        else:
            axes[idx, 0].text(0.5, 0.5, 'No good heating\ncurves found', 
                             ha='center', va='center', transform=axes[idx, 0].transAxes)
            axes[idx, 0].set_title(f'{room} - Heating')
        
        # Plot cooling curve
        if 'cooling_fits' in params and len(params['cooling_fits']) > 0:
            # Plot best cooling fit
            best_cooling = max(params['cooling_fits'], key=lambda x: x['r2'])
            
            axes[idx, 1].plot(best_cooling['time'], best_cooling['temp_actual'], 
                             'b-', linewidth=2, label='Actual')
            axes[idx, 1].plot(best_cooling['time'], best_cooling['temp_predicted'], 
                             'r--', linewidth=2, label='Model')
            
            axes[idx, 1].set_title(f'{room} - Cooling Curve\nτ={best_cooling["tau_hours"]:.2f}h, R²={best_cooling["r2"]:.3f}')
            axes[idx, 1].set_xlabel('Time (hours)')
            axes[idx, 1].set_ylabel('Temperature (°C)')
            axes[idx, 1].legend()
            axes[idx, 1].grid(True, alpha=0.3)
        else:
            axes[idx, 1].text(0.5, 0.5, 'No good cooling\ncurves found', 
                             ha='center', va='center', transform=axes[idx, 1].transAxes)
            axes[idx, 1].set_title(f'{room} - Cooling')
    
    plt.tight_layout()
    plt.show()

## 5. Thermal Resistance and Capacitance Estimation

Estimate thermal RC parameters from the time constants

In [ ]:
# Define room power ratings (should match those used in heating analysis)
ROOM_POWER_RATINGS = {
    'obyvacka': 2000,      # Living room - 2kW
    'kuchyn': 1500,        # Kitchen - 1.5kW  
    'loznice': 1500,       # Bedroom - 1.5kW
    'detsky_pokoj': 1000,  # Child room - 1kW
    'koupelna': 800,       # Bathroom - 800W
    'pracovna': 1200,      # Office - 1.2kW
    'chodba': 500,         # Hallway - 500W
    'spiz': 300,           # Pantry - 300W
}

def estimate_rc_parameters(room, heating_tau, cooling_tau, heating_power_w):
    """Estimate thermal resistance and capacitance from time constants."""
    
    # For RC circuit: tau = R * C
    # During heating: tau_heating ≈ R_total * C
    # During cooling: tau_cooling = R_loss * C
    # Where R_total includes heating resistance, R_loss is just thermal losses
    
    # Use cooling time constant (more reliable as it's natural decay)
    tau_primary = cooling_tau if cooling_tau > 0 else heating_tau
    
    if tau_primary <= 0:
        return None
    
    # Estimate thermal resistance from steady-state heat transfer
    # Assuming typical room temperature rise of 2-3°C during heating
    typical_temp_rise = 2.5  # °C
    
    # R = ΔT / P_heating (steady state)
    R_estimate = typical_temp_rise / heating_power_w  # K/W
    
    # C = tau / R
    C_estimate = tau_primary * 3600 / R_estimate  # J/K (convert hours to seconds)
    
    # Calculate derived parameters
    UA = 1 / R_estimate  # Heat loss coefficient W/K
    thermal_mass = C_estimate / 1000  # kJ/K
    
    return {
        'R_thermal': R_estimate,  # K/W
        'C_thermal': C_estimate,  # J/K
        'tau_primary': tau_primary,  # hours
        'UA_coefficient': UA,  # W/K
        'thermal_mass_kj_k': thermal_mass,  # kJ/K
        'heating_power': heating_power_w  # W
    }

# Calculate RC parameters for each room
room_rc_params = {}

for room in room_thermal_params.keys():
    params = room_thermal_params[room]
    heating_power = ROOM_POWER_RATINGS.get(room, 1000)  # Default 1kW if unknown
    
    heating_tau = params.get('heating_tau_avg', 0)
    cooling_tau = params.get('cooling_tau_avg', 0)
    
    rc_params = estimate_rc_parameters(room, heating_tau, cooling_tau, heating_power)
    
    if rc_params:
        room_rc_params[room] = rc_params

# Display RC parameters
if room_rc_params:
    print("\nThermal RC Parameters:")
    print("=" * 90)
    print(f"{'Room':15s} {'Power (W)':10s} {'R (K/W)':10s} {'C (MJ/K)':10s} {'τ (h)':8s} {'UA (W/K)':10s} {'Mass':8s}")
    print("-" * 90)
    
    for room, params in room_rc_params.items():
        R = params['R_thermal']
        C = params['C_thermal'] / 1e6  # Convert to MJ/K
        tau = params['tau_primary']
        UA = params['UA_coefficient']
        mass = params['thermal_mass_kj_k']
        power = params['heating_power']
        
        print(f"{room:15s} {power:10.0f} {R:10.4f} {C:10.2f} {tau:8.2f} {UA:10.1f} {mass:8.0f}")
    
    # Calculate summary statistics
    avg_R = np.mean([p['R_thermal'] for p in room_rc_params.values()])
    avg_C = np.mean([p['C_thermal'] for p in room_rc_params.values()]) / 1e6
    avg_tau = np.mean([p['tau_primary'] for p in room_rc_params.values()])
    total_thermal_mass = sum([p['thermal_mass_kj_k'] for p in room_rc_params.values()])
    
    print("-" * 90)
    print(f"{'AVERAGE':15s} {'-':10s} {avg_R:10.4f} {avg_C:10.2f} {avg_tau:8.2f} {'-':10s} {total_thermal_mass:8.0f}")
    
    print(f"\nSummary:")
    print(f"  Average thermal resistance: {avg_R:.4f} K/W")
    print(f"  Average thermal capacitance: {avg_C:.2f} MJ/K")
    print(f"  Average time constant: {avg_tau:.2f} hours")
    print(f"  Total building thermal mass: {total_thermal_mass:.0f} kJ/K")

## 6. Solar Gain Analysis

Analyze impact of solar radiation on room temperatures

In [ ]:
# Analyze solar gains for each room
solar_gain_analysis = {}

for room, room_data in thermal_datasets.items():
    if 'solar_power' in room_data.columns:
        # Filter daylight hours only
        daylight_data = room_data[(room_data['hour'] >= 6) & (room_data['hour'] <= 20)].copy()
        
        if len(daylight_data) > 100:  # Need sufficient data
            # Calculate temperature rate of change
            daylight_data['temp_rate'] = daylight_data['temperature'].diff() / 0.25  # °C per hour
            
            # Filter periods with no heating (to isolate solar effects)
            no_heating = daylight_data[daylight_data['heating_state'] < 0.1].copy()
            
            if len(no_heating) > 50:
                # Correlate solar power with temperature change
                solar_correlation = no_heating['solar_power'].corr(no_heating['temp_rate'])
                
                # Bin solar power and calculate average temperature rise
                solar_bins = pd.cut(no_heating['solar_power'], bins=10)
                temp_rise_by_solar = no_heating.groupby(solar_bins)['temp_rate'].mean()
                
                # Estimate solar gain coefficient
                # Linear regression: temp_rate = solar_gain_coeff * solar_power + baseline
                valid_data = no_heating[['solar_power', 'temp_rate']].dropna()
                
                if len(valid_data) > 20:
                    X = valid_data['solar_power'].values.reshape(-1, 1)
                    y = valid_data['temp_rate'].values
                    
                    reg = LinearRegression().fit(X, y)
                    solar_gain_coeff = reg.coef_[0]  # °C/h per W
                    baseline_temp_rate = reg.intercept_  # °C/h with no solar
                    r2_solar = reg.score(X, y)
                    
                    solar_gain_analysis[room] = {
                        'solar_correlation': solar_correlation,
                        'solar_gain_coefficient': solar_gain_coeff * 1000,  # °C/h per kW
                        'baseline_temp_rate': baseline_temp_rate,
                        'r2_solar_model': r2_solar,
                        'data_points': len(valid_data),
                        'temp_rise_by_solar': temp_rise_by_solar
                    }

# Display solar gain analysis
if solar_gain_analysis:
    print("\nSolar Gain Analysis:")
    print("=" * 70)
    print(f"{'Room':15s} {'Correlation':12s} {'Gain (°C/h/kW)':15s} {'R²':8s} {'Points':8s}")
    print("-" * 70)
    
    for room, analysis in solar_gain_analysis.items():
        corr = analysis['solar_correlation']
        gain = analysis['solar_gain_coefficient']
        r2 = analysis['r2_solar_model']
        points = analysis['data_points']
        
        print(f"{room:15s} {corr:11.3f} {gain:14.4f} {r2:7.3f} {points:7d}")
    
    # Find rooms with significant solar gains
    significant_solar_rooms = [room for room, analysis in solar_gain_analysis.items() 
                              if abs(analysis['solar_correlation']) > 0.3]
    
    if significant_solar_rooms:
        print(f"\nRooms with significant solar gains: {', '.join(significant_solar_rooms)}")
    else:
        print("\nNo rooms show significant solar gain correlation")
else:
    print("\nNo solar gain analysis possible (insufficient solar power data)")

## 7. Room-to-Room Heat Transfer Analysis

Analyze thermal coupling between adjacent rooms

In [ ]:
# Calculate cross-correlations between room temperatures
room_coupling_analysis = {}

room_list = list(thermal_datasets.keys())
if len(room_list) > 1:
    # Create temperature correlation matrix
    temp_data = pd.DataFrame()
    
    for room in room_list:
        room_data = thermal_datasets[room]
        temp_data[room] = room_data['temperature'].resample('H').mean()
    
    # Remove missing data
    temp_data = temp_data.dropna()
    
    if len(temp_data) > 24:  # Need at least 24 hours of data
        # Calculate correlation matrix
        correlation_matrix = temp_data.corr()
        
        # Calculate time-lagged correlations
        lag_correlations = {}
        max_lag_hours = 3
        
        for room1 in room_list:
            lag_correlations[room1] = {}
            for room2 in room_list:
                if room1 != room2:
                    # Calculate correlations at different lags
                    lags = range(-max_lag_hours, max_lag_hours + 1)
                    correlations = []
                    
                    for lag in lags:
                        if lag == 0:
                            corr = temp_data[room1].corr(temp_data[room2])
                        else:
                            shifted_data = temp_data[room2].shift(lag)
                            corr = temp_data[room1].corr(shifted_data)
                        correlations.append(corr)
                    
                    # Find best correlation and corresponding lag
                    best_idx = np.argmax(np.abs(correlations))
                    best_lag = lags[best_idx]
                    best_corr = correlations[best_idx]
                    
                    lag_correlations[room1][room2] = {
                        'best_correlation': best_corr,
                        'best_lag_hours': best_lag,
                        'all_correlations': correlations,
                        'lags': lags
                    }
        
        room_coupling_analysis = {
            'correlation_matrix': correlation_matrix,
            'lag_correlations': lag_correlations,
            'temp_data': temp_data
        }
        
        # Display correlation matrix
        print("\nRoom Temperature Correlation Matrix:")
        print("=" * 60)
        print(correlation_matrix.round(3))
        
        # Find strongest couplings
        strong_couplings = []
        for room1 in room_list:
            for room2 in room_list:
                if room1 != room2 and room1 in lag_correlations:
                    if room2 in lag_correlations[room1]:
                        corr_data = lag_correlations[room1][room2]
                        if abs(corr_data['best_correlation']) > 0.6:
                            strong_couplings.append({
                                'room1': room1,
                                'room2': room2,
                                'correlation': corr_data['best_correlation'],
                                'lag_hours': corr_data['best_lag_hours']
                            })
        
        if strong_couplings:
            print("\nStrong Room Couplings (|correlation| > 0.6):")
            print("-" * 60)
            print(f"{'Room 1':15s} {'Room 2':15s} {'Correlation':12s} {'Lag (h)':10s}")
            print("-" * 60)
            
            for coupling in strong_couplings:
                print(f"{coupling['room1']:15s} {coupling['room2']:15s} {coupling['correlation']:11.3f} {coupling['lag_hours']:9d}")
        else:
            print("\nNo strong room couplings found")
else:
    print("\nInsufficient rooms for coupling analysis")

## 8. Thermal Comfort Analysis

Analyze temperature stability and comfort metrics

In [ ]:
# Analyze thermal comfort for each room
thermal_comfort_analysis = {}

for room, room_data in thermal_datasets.items():
    # Calculate comfort metrics
    temp_series = room_data['temperature']
    
    # Basic statistics
    comfort_stats = {
        'mean_temp': temp_series.mean(),
        'std_temp': temp_series.std(),
        'min_temp': temp_series.min(),
        'max_temp': temp_series.max(),
        'temp_range': temp_series.max() - temp_series.min()
    }
    
    # Calculate temperature stability (variations)
    temp_diff = temp_series.diff().abs()
    comfort_stats['avg_temp_change'] = temp_diff.mean()
    comfort_stats['max_temp_change'] = temp_diff.max()
    
    # Calculate time spent in comfort zones
    comfort_zones = {
        'cold': (temp_series < 18).sum() / len(temp_series) * 100,  # Below 18°C
        'cool': ((temp_series >= 18) & (temp_series < 20)).sum() / len(temp_series) * 100,  # 18-20°C
        'comfortable': ((temp_series >= 20) & (temp_series <= 24)).sum() / len(temp_series) * 100,  # 20-24°C
        'warm': ((temp_series > 24) & (temp_series <= 26)).sum() / len(temp_series) * 100,  # 24-26°C
        'hot': (temp_series > 26).sum() / len(temp_series) * 100  # Above 26°C
    }
    
    comfort_stats['comfort_zones'] = comfort_zones
    
    # Calculate daily temperature patterns
    daily_temp_stats = temp_series.resample('D').agg(['min', 'max', 'mean', 'std'])
    comfort_stats['daily_range_avg'] = (daily_temp_stats['max'] - daily_temp_stats['min']).mean()
    comfort_stats['daily_range_max'] = (daily_temp_stats['max'] - daily_temp_stats['min']).max()
    
    # Heating efficiency (temperature rise per heating hour)
    if 'heating_state' in room_data.columns:
        heating_periods = room_data[room_data['heating_state'] > 0.5]
        if len(heating_periods) > 10:
            heating_temp_rise = heating_periods['temperature'].diff().mean()
            comfort_stats['heating_efficiency'] = heating_temp_rise * 4  # Convert to °C per hour
    
    thermal_comfort_analysis[room] = comfort_stats

# Display comfort analysis
if thermal_comfort_analysis:
    print("\nThermal Comfort Analysis:")
    print("=" * 80)
    print(f"{'Room':15s} {'Mean (°C)':10s} {'Std (°C)':9s} {'Range':8s} {'Comfort %':10s} {'Daily Range':12s}")
    print("-" * 80)
    
    for room, stats in thermal_comfort_analysis.items():
        mean_temp = stats['mean_temp']
        std_temp = stats['std_temp']
        temp_range = stats['temp_range']
        comfort_pct = stats['comfort_zones']['comfortable']
        daily_range = stats['daily_range_avg']
        
        print(f"{room:15s} {mean_temp:9.1f} {std_temp:8.2f} {temp_range:7.1f} {comfort_pct:9.1f} {daily_range:11.1f}")
    
    # Comfort zone summary
    print("\nComfort Zone Distribution:")
    print("-" * 70)
    print(f"{'Room':15s} {'Cold %':8s} {'Cool %':8s} {'Comfort %':10s} {'Warm %':8s} {'Hot %':7s}")
    print("-" * 70)
    
    for room, stats in thermal_comfort_analysis.items():
        zones = stats['comfort_zones']
        print(f"{room:15s} {zones['cold']:7.1f} {zones['cool']:7.1f} {zones['comfortable']:9.1f} {zones['warm']:7.1f} {zones['hot']:6.1f}")
    
    # Identify rooms with comfort issues
    comfort_issues = []
    for room, stats in thermal_comfort_analysis.items():
        if stats['comfort_zones']['comfortable'] < 70:
            comfort_issues.append(f"{room} (only {stats['comfort_zones']['comfortable']:.1f}% comfortable)")
        if stats['daily_range_avg'] > 4:
            comfort_issues.append(f"{room} (high daily variation: {stats['daily_range_avg']:.1f}°C)")
    
    if comfort_issues:
        print(f"\nRooms with potential comfort issues:")
        for issue in comfort_issues:
            print(f"  - {issue}")
    else:
        print(f"\nAll rooms appear to have good thermal comfort")

## 9. Visualize Thermal Analysis Results

Create comprehensive visualizations of thermal analysis

In [ ]:
# Create comprehensive thermal analysis visualizations
if room_rc_params and thermal_comfort_analysis:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Thermal time constants comparison
    rooms = list(room_rc_params.keys())
    heating_taus = [room_thermal_params[room].get('heating_tau_avg', 0) for room in rooms]
    cooling_taus = [room_thermal_params[room].get('cooling_tau_avg', 0) for room in rooms]
    
    x = np.arange(len(rooms))
    width = 0.35
    
    bars1 = axes[0,0].bar(x - width/2, heating_taus, width, label='Heating τ', alpha=0.8, color='red')
    bars2 = axes[0,0].bar(x + width/2, cooling_taus, width, label='Cooling τ', alpha=0.8, color='blue')
    
    axes[0,0].set_ylabel('Time Constant (hours)')
    axes[0,0].set_title('Thermal Time Constants by Room')
    axes[0,0].set_xticks(x)
    axes[0,0].set_xticklabels(rooms, rotation=45, ha='right')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Thermal resistance vs capacitance
    resistances = [room_rc_params[room]['R_thermal'] for room in rooms]
    capacitances = [room_rc_params[room]['C_thermal']/1e6 for room in rooms]  # MJ/K
    
    scatter = axes[0,1].scatter(resistances, capacitances, s=100, alpha=0.7)
    for i, room in enumerate(rooms):
        axes[0,1].annotate(room, (resistances[i], capacitances[i]), 
                          xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    axes[0,1].set_xlabel('Thermal Resistance (K/W)')
    axes[0,1].set_ylabel('Thermal Capacitance (MJ/K)')
    axes[0,1].set_title('Thermal R vs C')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Temperature comfort distribution
    comfort_data = []
    for room in rooms:
        if room in thermal_comfort_analysis:
            zones = thermal_comfort_analysis[room]['comfort_zones']
            comfort_data.append([zones['cold'], zones['cool'], zones['comfortable'], 
                               zones['warm'], zones['hot']])
    
    if comfort_data:
        comfort_df = pd.DataFrame(comfort_data, 
                                columns=['Cold', 'Cool', 'Comfortable', 'Warm', 'Hot'],
                                index=rooms)
        
        comfort_df.plot(kind='bar', stacked=True, ax=axes[0,2], 
                       color=['lightblue', 'blue', 'green', 'orange', 'red'])
        axes[0,2].set_ylabel('Time (%)')
        axes[0,2].set_title('Temperature Comfort Zones')
        axes[0,2].set_xticklabels(rooms, rotation=45, ha='right')
        axes[0,2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 4. Thermal mass vs heating power
    thermal_masses = [room_rc_params[room]['thermal_mass_kj_k'] for room in rooms]
    heating_powers = [room_rc_params[room]['heating_power']/1000 for room in rooms]  # kW
    
    scatter2 = axes[1,0].scatter(heating_powers, thermal_masses, s=100, alpha=0.7, color='purple')
    for i, room in enumerate(rooms):
        axes[1,0].annotate(room, (heating_powers[i], thermal_masses[i]), 
                          xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    axes[1,0].set_xlabel('Heating Power (kW)')
    axes[1,0].set_ylabel('Thermal Mass (kJ/K)')
    axes[1,0].set_title('Thermal Mass vs Heating Power')
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Temperature stability
    temp_stds = [thermal_comfort_analysis[room]['std_temp'] for room in rooms 
                if room in thermal_comfort_analysis]
    temp_ranges = [thermal_comfort_analysis[room]['daily_range_avg'] for room in rooms 
                  if room in thermal_comfort_analysis]
    
    if temp_stds and temp_ranges:
        bars = axes[1,1].bar(rooms, temp_stds, alpha=0.7, color='orange')
        axes[1,1].set_ylabel('Temperature Std Dev (°C)')
        axes[1,1].set_title('Temperature Stability')
        axes[1,1].set_xticklabels(rooms, rotation=45, ha='right')
        axes[1,1].grid(True, alpha=0.3)
    
    # 6. Room coupling heatmap
    if 'correlation_matrix' in room_coupling_analysis:
        corr_matrix = room_coupling_analysis['correlation_matrix']
        im = axes[1,2].imshow(corr_matrix.values, cmap='coolwarm', vmin=-1, vmax=1)
        axes[1,2].set_xticks(range(len(corr_matrix.columns)))
        axes[1,2].set_yticks(range(len(corr_matrix.index)))
        axes[1,2].set_xticklabels(corr_matrix.columns, rotation=45, ha='right')
        axes[1,2].set_yticklabels(corr_matrix.index)
        axes[1,2].set_title('Room Temperature Correlations')
        
        # Add correlation values
        for i in range(len(corr_matrix.index)):
            for j in range(len(corr_matrix.columns)):
                text = axes[1,2].text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                                     ha="center", va="center", color="black", fontsize=8)
        
        plt.colorbar(im, ax=axes[1,2])
    else:
        axes[1,2].text(0.5, 0.5, 'No room coupling\ndata available', 
                      ha='center', va='center', transform=axes[1,2].transAxes)
        axes[1,2].set_title('Room Coupling')
    
    plt.tight_layout()
    plt.show()

## 10. Thermal Model Validation

Validate thermal models against actual temperature data

In [ ]:
def validate_thermal_model(room_data, rc_params, validation_hours=24):
    """Validate thermal model against actual data."""
    
    if len(room_data) < validation_hours * 4:  # Need enough data (15-min intervals)
        return None
    
    # Use last portion of data for validation
    validation_data = room_data.tail(validation_hours * 4).copy()
    
    # Extract parameters
    R = rc_params['R_thermal']
    C = rc_params['C_thermal']
    P_heating = rc_params['heating_power']
    
    # Initial conditions
    T_initial = validation_data['temperature'].iloc[0]
    
    # Simulate temperature using RC model
    dt = 0.25  # 15 minutes = 0.25 hours
    temperatures_sim = [T_initial]
    
    for i in range(1, len(validation_data)):
        T_prev = temperatures_sim[-1]
        heating_state = validation_data['heating_state'].iloc[i]
        
        # Assume outdoor temperature (use weather data if available)
        if 'temperature_2m' in validation_data.columns:
            T_outdoor = validation_data['temperature_2m'].iloc[i]
        else:
            T_outdoor = 15  # Default outdoor temperature
        
        # Solar gains (if available)
        solar_gain = 0
        if 'solar_power' in validation_data.columns:
            solar_gain = validation_data['solar_power'].iloc[i] * 0.001  # Simple conversion
        
        # RC differential equation: C * dT/dt = P_heating - (T - T_outdoor)/R + P_solar
        P_total = heating_state * P_heating + solar_gain
        heat_loss = (T_prev - T_outdoor) / R
        
        dT_dt = (P_total - heat_loss) / C  # °C per second
        T_new = T_prev + dT_dt * dt * 3600  # Convert to hours
        
        temperatures_sim.append(T_new)
    
    # Calculate validation metrics
    T_actual = validation_data['temperature'].values
    T_simulated = np.array(temperatures_sim)
    
    rmse = np.sqrt(mean_squared_error(T_actual, T_simulated))
    mae = np.mean(np.abs(T_actual - T_simulated))
    r2 = r2_score(T_actual, T_simulated)
    
    return {
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'T_actual': T_actual,
        'T_simulated': T_simulated,
        'time_index': validation_data.index
    }

# Validate thermal models
model_validation_results = {}

for room in room_rc_params.keys():
    if room in thermal_datasets:
        room_data = thermal_datasets[room]
        rc_params = room_rc_params[room]
        
        validation_result = validate_thermal_model(room_data, rc_params)
        
        if validation_result:
            model_validation_results[room] = validation_result

# Display validation results
if model_validation_results:
    print("\nThermal Model Validation Results:")
    print("=" * 60)
    print(f"{'Room':15s} {'RMSE (°C)':10s} {'MAE (°C)':10s} {'R²':8s} {'Status':10s}")
    print("-" * 60)
    
    for room, results in model_validation_results.items():
        rmse = results['rmse']
        mae = results['mae']
        r2 = results['r2']
        
        # Determine model quality
        if r2 > 0.8 and rmse < 1.0:
            status = "Excellent"
        elif r2 > 0.6 and rmse < 1.5:
            status = "Good"
        elif r2 > 0.4 and rmse < 2.0:
            status = "Fair"
        else:
            status = "Poor"
        
        print(f"{room:15s} {rmse:9.3f} {mae:9.3f} {r2:7.3f} {status:10s}")
    
    # Plot validation results for best performing room
    best_room = max(model_validation_results.keys(), 
                   key=lambda x: model_validation_results[x]['r2'])
    
    best_results = model_validation_results[best_room]
    
    plt.figure(figsize=(12, 6))
    plt.plot(best_results['time_index'], best_results['T_actual'], 
             'b-', linewidth=2, label='Actual', alpha=0.8)
    plt.plot(best_results['time_index'], best_results['T_simulated'], 
             'r--', linewidth=2, label='Model', alpha=0.8)
    
    plt.xlabel('Time')
    plt.ylabel('Temperature (°C)')
    plt.title(f'Thermal Model Validation - {best_room}\n'
             f'RMSE: {best_results["rmse"]:.3f}°C, R²: {best_results["r2"]:.3f}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    avg_r2 = np.mean([r['r2'] for r in model_validation_results.values()])
    avg_rmse = np.mean([r['rmse'] for r in model_validation_results.values()])
    
    print(f"\nOverall Model Performance:")
    print(f"  Average R²: {avg_r2:.3f}")
    print(f"  Average RMSE: {avg_rmse:.3f}°C")
    
    good_models = sum(1 for r in model_validation_results.values() 
                     if r['r2'] > 0.6 and r['rmse'] < 1.5)
    print(f"  Good models: {good_models}/{len(model_validation_results)}")
    
else:
    print("\nNo thermal model validation possible (insufficient data)")

## 11. Summary and Recommendations

Generate actionable insights and thermal optimization recommendations

In [ ]:
print("\nThermal Analysis - Key Insights and Recommendations:")
print("=" * 80)

# Generate insights based on thermal analysis
insights = []
recommendations = []

# Thermal time constant insights
if room_thermal_params:
    fast_rooms = []
    slow_rooms = []
    
    for room, params in room_thermal_params.items():
        if 'heating_tau_avg' in params:
            tau = params['heating_tau_avg']
            if tau < 2:
                fast_rooms.append((room, tau))
            elif tau > 6:
                slow_rooms.append((room, tau))
    
    if fast_rooms:
        insights.append(f"Fast-response rooms: {', '.join([f'{r}({t:.1f}h)' for r, t in fast_rooms])}")
        recommendations.append("Use fast-response rooms for quick heating adjustments")
    
    if slow_rooms:
        insights.append(f"Slow-response rooms: {', '.join([f'{r}({t:.1f}h)' for r, t in slow_rooms])}")
        recommendations.append("Pre-heat slow-response rooms before occupancy periods")

# Thermal efficiency insights
if room_rc_params:
    inefficient_rooms = []
    efficient_rooms = []
    
    for room, params in room_rc_params.items():
        R = params['R_thermal']
        if R > 0.002:  # High thermal resistance (poor insulation)
            inefficient_rooms.append((room, R))
        elif R < 0.001:  # Low thermal resistance (good insulation)
            efficient_rooms.append((room, R))
    
    if inefficient_rooms:
        insights.append(f"Thermally inefficient rooms: {', '.join([f'{r}({R:.4f}K/W)' for r, R in inefficient_rooms])}")
        recommendations.append("Consider insulation improvements for thermally inefficient rooms")
    
    if efficient_rooms:
        insights.append(f"Thermally efficient rooms: {', '.join([f'{r}({R:.4f}K/W)' for r, R in efficient_rooms])}")

# Comfort insights
if thermal_comfort_analysis:
    comfort_issues = []
    stable_rooms = []
    
    for room, stats in thermal_comfort_analysis.items():
        comfort_pct = stats['comfort_zones']['comfortable']
        temp_std = stats['std_temp']
        
        if comfort_pct < 70:
            comfort_issues.append((room, comfort_pct))
        elif temp_std < 0.5:
            stable_rooms.append((room, temp_std))
    
    if comfort_issues:
        insights.append(f"Rooms with comfort issues: {', '.join([f'{r}({p:.1f}%)' for r, p in comfort_issues])}")
        recommendations.append("Review heating schedules and setpoints for rooms with poor comfort")
    
    if stable_rooms:
        insights.append(f"Thermally stable rooms: {', '.join([f'{r}({s:.2f}°C)' for r, s in stable_rooms])}")

# Solar gain insights
if solar_gain_analysis:
    high_solar_rooms = [(room, analysis['solar_gain_coefficient']) 
                       for room, analysis in solar_gain_analysis.items() 
                       if analysis['solar_gain_coefficient'] > 0.5]
    
    if high_solar_rooms:
        insights.append(f"High solar gain rooms: {', '.join([f'{r}({g:.2f}°C/h/kW)' for r, g in high_solar_rooms])}")
        recommendations.append("Utilize solar gains for passive heating in high solar gain rooms")
        recommendations.append("Consider automated shading for overheating prevention")

# Model validation insights
if model_validation_results:
    good_models = [(room, results['r2']) for room, results in model_validation_results.items() 
                  if results['r2'] > 0.7]
    poor_models = [(room, results['r2']) for room, results in model_validation_results.items() 
                  if results['r2'] < 0.5]
    
    if good_models:
        insights.append(f"Rooms with good thermal models: {', '.join([f'{r}(R²={r2:.2f})' for r, r2 in good_models])}")
        recommendations.append("Use validated thermal models for predictive control")
    
    if poor_models:
        insights.append(f"Rooms with poor thermal models: {', '.join([f'{r}(R²={r2:.2f})' for r, r2 in poor_models])}")
        recommendations.append("Investigate thermal modeling issues in rooms with poor model fit")

# Room coupling insights
if 'strong_couplings' in locals() and strong_couplings:
    insights.append(f"Strong room thermal coupling detected in {len(strong_couplings)} pairs")
    recommendations.append("Consider coordinated heating control for thermally coupled rooms")

# Display insights and recommendations
print("\nKey Insights:")
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\nOptimization Recommendations:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

# Summary statistics
print("\nSummary Statistics:")
print("-" * 40)
if room_rc_params:
    avg_tau = np.mean([params.get('heating_tau_avg', 0) for params in room_thermal_params.values() 
                      if 'heating_tau_avg' in params])
    total_thermal_mass = sum([params['thermal_mass_kj_k'] for params in room_rc_params.values()])
    avg_R = np.mean([params['R_thermal'] for params in room_rc_params.values()])
    
    print(f"Average thermal time constant: {avg_tau:.2f} hours")
    print(f"Total building thermal mass: {total_thermal_mass:.0f} kJ/K")
    print(f"Average thermal resistance: {avg_R:.4f} K/W")

if thermal_comfort_analysis:
    avg_comfort = np.mean([stats['comfort_zones']['comfortable'] 
                          for stats in thermal_comfort_analysis.values()])
    print(f"Average comfort time: {avg_comfort:.1f}%")

print(f"\nThermal analysis completed for {len(thermal_datasets)} rooms.")

In [ ]:
# Save thermal analysis results
import pickle
from pathlib import Path

# Create comprehensive results dictionary
thermal_analysis_results = {
    'analysis_period': {'start': start_date, 'end': end_date},
    'rooms_analyzed': list(thermal_datasets.keys()),
    'thermal_parameters': room_thermal_params,
    'rc_parameters': room_rc_params,
    'comfort_analysis': thermal_comfort_analysis,
    'model_validation': model_validation_results,
    'insights': insights,
    'recommendations': recommendations
}

# Add optional analyses if available
if solar_gain_analysis:
    thermal_analysis_results['solar_gain_analysis'] = solar_gain_analysis

if room_coupling_analysis:
    thermal_analysis_results['room_coupling_analysis'] = {
        'correlation_matrix': room_coupling_analysis['correlation_matrix'].to_dict(),
        'strong_couplings': strong_couplings if 'strong_couplings' in locals() else []
    }

# Save to files
results_dir = Path('../../../data/processed')
results_dir.mkdir(parents=True, exist_ok=True)

# Save as pickle for programmatic use
with open(results_dir / 'thermal_analysis_results.pkl', 'wb') as f:
    pickle.dump(thermal_analysis_results, f)

# Save thermal parameters as CSV for easy viewing
if room_rc_params:
    rc_df = pd.DataFrame.from_dict(room_rc_params, orient='index')
    rc_df.to_csv(results_dir / 'thermal_rc_parameters.csv')

# Save summary as text
with open(results_dir / 'thermal_analysis_summary.txt', 'w') as f:
    f.write("Thermal Dynamics Analysis Summary\n")
    f.write("=" * 40 + "\n\n")
    f.write(f"Analysis Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}\n")
    f.write(f"Rooms Analyzed: {len(thermal_datasets)}\n\n")
    
    f.write("Key Insights:\n")
    for i, insight in enumerate(insights, 1):
        f.write(f"{i}. {insight}\n")
    
    f.write("\nRecommendations:\n")
    for i, rec in enumerate(recommendations, 1):
        f.write(f"{i}. {rec}\n")
    
    if room_rc_params:
        f.write("\nThermal RC Parameters:\n")
        f.write(f"{'Room':15s} {'R (K/W)':10s} {'C (MJ/K)':10s} {'τ (h)':8s}\n")
        f.write("-" * 50 + "\n")
        for room, params in room_rc_params.items():
            R = params['R_thermal']
            C = params['C_thermal'] / 1e6
            tau = params['tau_primary']
            f.write(f"{room:15s} {R:10.4f} {C:10.2f} {tau:8.2f}\n")

print("\nThermal analysis results saved to:")
print(f"  - {results_dir / 'thermal_analysis_results.pkl'}")
print(f"  - {results_dir / 'thermal_rc_parameters.csv'}")
print(f"  - {results_dir / 'thermal_analysis_summary.txt'}")